### **라이브러리 호출**

In [160]:
import pandas as pd
import numpy as np
from haversine import haversine, haversine_vector
from geopy.geocoders import Nominatim
import ast

In [161]:
apartment_data = pd.read_csv('청약홈_합본.csv', encoding='cp949').drop('Unnamed: 0', axis=1)
apartment = apartment_data.copy()
apartment.columns

Index(['아파트명', '면적', '법정동주소', '도로명주소', '위도', '경도', '세대수', '임대세대수', '영구임대세대수',
       '국민임대세대수', '민간임대세대수', '공공임대세대수', '장기임대세대수', '기타임대세대수', '최고층', '최저층',
       '최대공급면적', '최소공급면적', '총아파트동수', '용적률', '건폐율', '주차대수', '세대평균_주차대수', '건설사',
       '공급면적', '전용면적', '전용율', '방수', '욕실수', '해당면적_세대수', '현관구조', '가격',
       '초등학교_학군정보', '초등학교_설립정보', '입주예정일', '공급액(만원)'],
      dtype='object')

중복되는 컬럼 제거

In [162]:
apartment.drop_duplicates(inplace=True)

In [163]:
apartment.shape

(3345, 36)

입주예정일을 입주예정연도로 변경

In [164]:
apartment['입주예정연도'] = apartment['입주예정일'].apply(lambda x: str(x).replace("-","")[0:4])

In [165]:
apartment['location'] = tuple(zip(apartment['위도'], apartment['경도']))
apartment_location = list(apartment['location'])

# 변수 추가

* 대형건설사 시공 여부
* 가까운 지하철 역과 그 거리, 해당 노선
* 주변 의료기관 개수
* 주변 공원 개수
* 주변 대학 개수
* 광역/기초
* 주변 상권
* 평당 공급액

### **1. 대형건설사**

* 대형건설사가 시공에 참여한 아파트를 구분
* '대형건설사' 기준: 10대 건설사(삼성물산, 현대건설, DL이앤씨, 포스코이앤씨, GS건설, 대우건설, 현대엔지니어링, 롯데건설, SK에코플랜트, HDC현대산업개발)에 포함


In [166]:
# 숫자, ., 탭, ㈜, 주식회사 등 제거 

apartment['건설사'] = apartment['건설사'].str.replace("[0-9.\s]|\t|\(주\)|㈜|주\)|주식회사", "", regex=True)

In [167]:
# 건설사 이름 통일

def replace_firm_name(firm):
    if '현대산업개발' in firm:
        firm = firm.replace('HDC현대산업개발', '현대산업개발')
        firm = firm.replace('에이치디씨현대산업개발', '현대산업개발')
        firm = firm.replace('현대산업개발', 'HDC현대산업개발')
    if '디엘이앤씨' in firm:
        firm = firm.replace('디엘이앤씨', 'DL이앤씨') 
    if '지에스건설' in firm:
        firm = firm.replace('지에스건설', 'GS건설')
    if '에스케이에코플랜트' in firm:
        firm = firm.replace('에스케이에코플랜트', 'SK에코플랜트')
    if '에스케이건설' in firm:
        firm = firm.replace('에스케이건설', 'SK에코플랜트') # 사명 변경
    if '포스코건설' in firm:
        firm = firm.replace('포스코건설', '포스코이앤씨')   # 사명 변경
    
    return firm

apartment['건설사'] = apartment['건설사'].apply(replace_firm_name)

In [168]:
# 대형 건설사가 시공에 포함됐을 경우 '대형건설사' 컬럼 값 True

대형건설사 = ['삼성물산', '현대건설', 'DL이앤씨', '포스코이앤씨', 'GS건설', '대우건설', '현대엔지니어링', '롯데건설', 'SK에코플랜트', 'HDC현대산업개발']
apartment['대형건설사'] = apartment['건설사'].apply(lambda x: True if any(keyword in x for keyword in 대형건설사) else False)

### **2. 가장 가까운 지하철역명과 그 역과의 거리**

* 도시철도 역사정보 데이터: https://data.kric.go.kr/rips/M_01_01/detail.do;jsessionid=1jTd7vFOC+oGNYyEJtzQH6i1?id=32&lcd=A

* 서울 지하철역 좌표 데이터(보완/검토용): https://observablehq.com/@taekie/seoul_subway_station_coordinate

In [169]:
all_rail = pd.read_excel("전체_도시철도역사정보_20230630.xlsx")

In [170]:
# 서울, 인천, 경기지역 지하철만 필터링

cities_to_remove = ['대구', '부산', '대전', '광주광역시', '울산'] # '광주'만 넣을 경우 '경기도 광주' 포함

# 기관명 또는 주소에 필터링 적용
greater_seoul_rail = ~(all_rail['운영기관명'].str.contains('|'.join(cities_to_remove)) | all_rail['역사도로명주소'].str.contains('|'.join(cities_to_remove)))
rail = all_rail[greater_seoul_rail]

In [171]:
# 괄호와 괄호글, 공백, 탭, .,  줄바꿈 등 제거

rail['역사명'] = rail['역사명'].str.replace(r"\s|\n|\([^()]*\)|[.·]", "", regex=True)

# '역'으로 끝나지 않을 경우 역을 붙여줌
rail['역사명'] = rail['역사명'].apply(lambda x: x + '역' if not x.endswith('역') else x)

C:\Users\henna\AppData\Local\Temp\ipykernel_1144\2897763605.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rail['역사명'] = rail['역사명'].str.replace(r"\s|\n|\([^()]*\)|[.·]", "", regex=True)
C:\Users\henna\AppData\Local\Temp\ipykernel_1144\2897763605.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rail['역사명'] = rail['역사명'].apply(lambda x: x + '역' if not x.endswith('역') else x)


In [172]:
rail = rail[rail['노선명'] != '자기부상철도'] # 인천공항 자기부상열차는 무기한 운행 중단

In [173]:
# station 데이터로 보완

station = pd.read_csv("station_coordinate.csv")
station['name'] = station['name'] + '역'
station['name'] = station['name'].str.replace("[.,]", "", regex=True)
station_name = set(station['name'].unique())

rail_name = set(rail['역사명'].unique())

In [174]:
list(station_name - rail_name)

['이수역', '원곡역']

In [175]:
# 총신대입구(이수)역은 4호선과 7호선 2개 역
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '4호선'), '역위도'] = station[station['name'] == '총신대입구역']['lat'].values
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '4호선'), '역경도'] = station[station['name'] == '총신대입구역']['lng'].values
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '7호선'), '역위도'] = station[station['name'] == '이수역']['lat'].values
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '7호선'), '역경도'] = station[station['name'] == '이수역']['lng'].values

In [176]:
# '원곡역'은 '시우역'으로 역사명이 변경되었음(rail데이터에 있어 무관)
rail[rail['역사명'] == '시우역']

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
1041,4814,시우역,I41WS,서해선,Siu,時雨,일반역,NaN,NaN,37.313767,126.798303,한국철도공사,경기도 안산시 단원구 동산로 지하 50,1544-7788,2022-08-31


In [177]:
merged_rail = rail.merge(station[['name', 'lat', 'lng']], left_on = '역사명', right_on='name', how='left')

merged_rail['역위도'] = merged_rail['역위도'].fillna(merged_rail['lat'])
merged_rail['역경도'] = merged_rail['역경도'].fillna(merged_rail['lng'])

merged_rail.drop(['name', 'lat', 'lng'], axis=1, inplace=True)
merged_rail = merged_rail.drop_duplicates()

In [178]:
# station 데이터로 보완했음에도 불구하고 위경도가 없는 행들 확인

blank_df = merged_rail[merged_rail['역위도'].isna() | merged_rail['역경도'].isna()]

In [179]:
# 위경도가 없는 행들은 Nominatime 사용 주소로 위경도를 불러오길 시도
def geocoding(address):
    geo_local = Nominatim(user_agent='South Korea')
    location = geo_local.geocode(address)
    if location is not None and location.latitude is not None and location.longitude is not None:
        geo = (location.latitude, location.longitude)
        return geo
    else:
        return (0, 0)
    
blank_df.loc[:, '역위도'] = blank_df['역사도로명주소'].apply(lambda x: geocoding(x)[0])
blank_df.loc[:, '역경도'] = blank_df['역사도로명주소'].apply(lambda x: geocoding(x)[1])

In [180]:
# 그러나 여전히 업데이트 되지 않는 역들 존재재

blank_df[(blank_df['역위도'] == 0)|(blank_df['역경도'] == 0)][['역사명', '역사도로명주소']]

,역사명,역사도로명주소
75,산곡역,인천광역시 부평구 길주로 지하 379 (산곡동)
186,오남역,경기도 남양주시 오남읍 진건오남로 지하 929
453,강일역,서울특별시 강동구 고덕로 지하456(강일동)
454,미사역,경기도 하남시 미사강변동로 지하90(망월동)
455,하남풍산역,경기도 하남시 덕풍서로 지하50(덕풍동)
456,하남시청역,경기도 하남시 하남대로 지하820(덕풍동)
457,하남검단산역,경기도 하남시 대청로 지하100(창우동)
587,남위례역,경기도 성남시 수정구 공원로 601(복정동)
637,삼전역,서울특별시 송파구 백제고분로 지하 187
638,석촌고분역,서울특별시 송파구 삼학사로 지하 53


geocoding 함수가 적용되지 않는 것들은 데이터상 주소의 문제 -> 주소 수정

In [181]:
address_updates = [
    ('산곡역', '인천광역시 부평구 산곡동 10-32'),
    ('오남역', '경기도 남양주시 오남읍 진건오남로 929'),
    ('강일역', '서울특별시 강동구 강일동 산22-14'),
    ('미사역', '경기도 하남시 망월동 109-8'),
    ('하남풍산역', '경기도 하남시 덕풍동 727-3'),
    ('하남시청역', '경기도 하남시 신장동 510-2'),
    ('하남검단산역', '경기도 하남시 창우동 526'),
    ('남위례역', '경기도 성남시 수정구 복정동 57'),
    ('삼전역', '서울특별시 송파구 잠실동 347'),
    ('석촌고분역', '서울특별시 송파구 석촌동 157'),
    ('송파나루역', '서울특별시 송파구 송파동 3'),
    ('한성백제역', '서울특별시 송파구 방이동 88-17'),
    ('둔촌오륜역', '서울특별시 강동구 강동대로 303'),
    ('중앙보훈병원역', '서울특별시 강동구 둔촌동 8-1')]

for station_name, address in address_updates:
    blank_df.loc[blank_df['역사명'] == station_name, '역사도로명주소'] = address

In [182]:
blank_df.loc[:, '역위도'] = blank_df['역사도로명주소'].apply(lambda x: geocoding(x)[0])
blank_df.loc[:, '역경도'] = blank_df['역사도로명주소'].apply(lambda x: geocoding(x)[1])

In [183]:
# 이제 위도 경도가 없는 역은 없음을 확인
blank_df[(blank_df['역위도'] == 0)|(blank_df['역경도'] == 0)][['역사명', '역사도로명주소']]

,역사명,역사도로명주소


In [184]:
# blank_df값으로 merged_rail 업데이트

for index, data in merged_rail.iterrows():
    if data['역사명'] in blank_df['역사명'].values:
        new_info = blank_df.loc[blank_df['역사명'] == data['역사명']]
        merged_rail.at[index, '역위도'] = new_info['역위도'].values[0]
        merged_rail.at[index, '역경도'] = new_info['역경도'].values[0]

In [185]:
# 재확인
merged_rail.loc[(merged_rail['역위도'] == 0) | (merged_rail['역경도'] == 0)]

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자


'역사명'이 다른데 '역위도'와 '역경도'가 같은 행 존재 -> 주소 수정 필요

In [186]:
duplicated = merged_rail[merged_rail.duplicated(subset=['역위도', '역경도'], keep=False) & 
                         ~merged_rail.duplicated(subset=['역위도', '역경도', '역사명'], keep=False)
                        ]
duplicated_sorted = duplicated.sort_values(by=['역위도'])
duplicated_sorted

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
716,1701,구로역,I4101,경부선,Guro,九 老,일반역,NaN,NaN,37.503178,126.882037,한국철도공사,서울시 구로구 구로중앙로 174,1544-7788,2022-08-31
814,1205,구리역,I4108,경의중앙선,Guri,九 里,일반역,NaN,NaN,37.503178,126.882037,한국철도공사,경기도 구리시 건원대로 34번길 32-29(인창동 244-2),1544-7788,2022-08-31
177,G105,걸포북변역,L41G1,김포골드라인,Geolpo Bukbyeon,杰浦北边,일반역,NaN,NaN,37.614886,126.728671,김포골드라인운영㈜,경기도 김포시 김포대로 1040,031-8048-1750,2023-05-26 00:00:00
178,G106,사우역,L41G1,김포골드라인,Sau(Gimpo City Hall),萨乌,일반역,NaN,NaN,37.614886,126.728671,김포골드라인운영㈜,경기도 김포시 김포대로 852,031-8048-1760,2023-05-26 00:00:00
179,G107,풍무역,L41G1,김포골드라인,Pungmu,风舞,일반역,NaN,NaN,37.614886,126.728671,김포골드라인운영㈜,경기도 김포시 김포대로 710,031-8048-1770,2023-05-26 00:00:00
784,1268,화전역,I4108,경의중앙선,Hwajeon,花 田,일반역,NaN,NaN,37.637837,126.832503,한국철도공사,경기도 고양시 덕양구 화랑로 53(화전동 183-10),1544-7788,2022-08-31
1016,1952,화정역,I4106,일산선,Hwajeong,花 井,일반역,NaN,NaN,37.637837,126.832503,한국철도공사,경기도 고양시 덕양구 화정동 400-2,1544-7788,2022-08-31
175,G103,장기역,L41G1,김포골드라인,Janggi,象棋,일반역,NaN,NaN,37.640823,126.666295,김포골드라인운영㈜,경기도 김포시 김포한강1로 59,031-8048-1730,2023-05-26 00:00:00
176,G104,운양역,L41G1,김포골드라인,Unyang,云阳,일반역,NaN,NaN,37.640823,126.666295,김포골드라인운영㈜,경기도 김포시 김포한강1로 235,031-8048-1740,2023-05-26 00:00:00


In [187]:
address_updates = [
    ('구로역', '서울특별시 구로구 구로동 585-3'),
    ('구리역', '경기도 구리시 인창동 244-2'),
    ('걸포북변역', '경기도 김포시 북변동 135-10'),
    ('사우역', '경기도 김포시 사우동 854'),
    ('풍무역', '경기도 김포시 김포대로 710'),
    ('화전역', '경기도 고양시 덕양구 화전동 183-10'),
    ('화정역', '경기도 고양시 덕양구 화정동 1098'),
    ('장기역', '경기도 김포시 장기동 1791'),
    ('운양역', '경기도 김포시 운양동 1403')]

for station_name, address in address_updates:
    duplicated.loc[duplicated['역사명'] == station_name, '역사도로명주소'] = address
    
duplicated.loc[:, '역위도'] = duplicated['역사도로명주소'].apply(lambda x: geocoding(x)[0])
duplicated.loc[:, '역경도'] = duplicated['역사도로명주소'].apply(lambda x: geocoding(x)[1])

In [188]:
# 업데이트된 duplicated로 merged_rail 업데이트

for index, data in merged_rail.iterrows():
    if data['역사명'] in duplicated['역사명'].values:
        new_info = duplicated.loc[duplicated['역사명'] == data['역사명']]
        merged_rail.at[index, '역위도'] = new_info['역위도'].values[0]
        merged_rail.at[index, '역경도'] = new_info['역경도'].values[0]

In [189]:
# 역위도와 역경도가 같은데 역사명은 다른 컬럼 존재 있는지 재확인 -> 없음 
still_duplicated = merged_rail[merged_rail.duplicated(subset=['역위도', '역경도'], keep=False) & 
                         ~merged_rail.duplicated(subset=['역위도', '역경도', '역사명'], keep=False)]
still_duplicated

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자


노선명 정리

In [190]:
merged_rail['노선명'].unique()

array(['인천지하철 1호선', '인천지하철 2호선', '도시철도 7호선', '에버라인', '인천국제공항선', '우이신설선',
       '신분당선', '수도권 경량도시철도 신림선', '김포골드라인', '진접선', '1호선', '2호선', '3호선',
       '4호선', '5호선', '6호선', '7호선', '8호선', '수도권  도시철도 9호선', '의정부', '경강선',
       '경부선', '경원선', '경의중앙선', '경인선', '경춘선', '분당선', '수인선', '안산과천선', '일산선',
       '서해선'], dtype=object)

In [191]:
# 명칭 통일, 위경도 바뀐 데이터 수정
merged_rail.loc[merged_rail['노선명'] == '도시철도 7호선', '노선명'] = '7호선'
merged_rail.loc[merged_rail['노선명'] == '수도권  도시철도 9호선', '노선명'] = '9호선'
merged_rail.loc[merged_rail['노선명'] == '의정부', '노선명'] = '의정부선'
merged_rail.loc[merged_rail['노선명'] == '일산선', '노선명'] = '3호선'
merged_rail.loc[merged_rail['노선명'] == '수도권 경량도시철도 신림선', '노선명'] = '신림선'
merged_rail.loc[merged_rail['노선명'] == '경부선', '노선명'] = '1호선'
merged_rail.loc[merged_rail['노선명'] == '경인선', '노선명'] = '1호선'
merged_rail.loc[merged_rail['노선명'] == '경원선', '노선명'] = '1호선'
merged_rail.loc[merged_rail['노선명'] == '안산과천선', '노선명'] = '4호선'

errored = merged_rail['노선명'] == '신분당선'
merged_rail.loc[errored, ['역위도', '역경도']] = merged_rail.loc[errored, ['역경도', '역위도']].values

In [192]:
merged_rail['노선명'].unique()

array(['인천지하철 1호선', '인천지하철 2호선', '7호선', '에버라인', '인천국제공항선', '우이신설선',
       '신분당선', '신림선', '김포골드라인', '진접선', '1호선', '2호선', '3호선', '4호선', '5호선',
       '6호선', '8호선', '9호선', '의정부선', '경강선', '경의중앙선', '경춘선', '분당선', '수인선',
       '서해선'], dtype=object)

In [193]:
merged_rail['location'] = tuple(zip(merged_rail['역위도'], merged_rail['역경도']))
merged_rail['subway'] = merged_rail['노선명'] + ' ' + merged_rail['역사명']

In [194]:
merged_rail.to_csv("subway.csv", encoding='UTF-8', index=False)

### **수집한 데이터 오류 수정**

* 청약 신도시 사업이 끝난 후 주소를 정상 발급된 지번(도로명)주소로 업데이트
* 결측값 보완

In [195]:
fixed = pd.read_csv("fixed_20230829.csv")

In [196]:
fixed.columns

Index(['아파트명', '법정동주소', '위도', '경도', '세대수', '임대세대수', '최고층', '최저층', '최대공급면적',
       '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적', '전용면적', '전용율',
       '방수', '욕실수', '현관구조', '입주예정연도', '공급액(만원)', '대형건설사'],
      dtype='object')

지하철역

In [197]:
merged_rail.head()

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자,location,subway
0,3110,계양역,S2801,인천지하철 1호선,Gyeyang,桂陽,환승역,S2801+I28A1,공항철도,37.571539,126.736319,인천교통공사,인천광역시 계양구 다남로 24,032-710-9105,2022-04-30 00:00:00,"(37.571539, 126.736319)",인천지하철 1호선 계양역
2,3111,귤현역,S2801,인천지하철 1호선,Gyulhyeon,橘峴,일반역,-,-,37.566362,126.742498,인천교통공사,인천광역시 계양구 장제로 1136,032-515-9104,2022-04-30 00:00:00,"(37.566362, 126.742498)",인천지하철 1호선 귤현역
3,3112,박촌역,S2801,인천지하철 1호선,Bakchon,朴村,일반역,-,-,37.553525,126.744946,인천교통공사,인천광역시 계양구 장제로 992,032-519-3122,2022-04-30 00:00:00,"(37.553525, 126.744946)",인천지하철 1호선 박촌역
4,3113,임학역,S2801,인천지하철 1호선,Imhak,林鶴,일반역,-,-,37.545058,126.738642,인천교통공사,인천광역시 계양구 장제로 875,032-541-3113,2022-04-30 00:00:00,"(37.545058, 126.738642)",인천지하철 1호선 임학역
5,3114,계산역,S2801,인천지하철 1호선,Gyesan,桂山,일반역,-,-,37.543243,126.728436,인천교통공사,인천광역시 계양구 경명대로 1089,032-546-3151,2022-04-30 00:00:00,"(37.543243, 126.728436)",인천지하철 1호선 계산역


In [198]:
fixed['location'] = tuple(zip(fixed['위도'], fixed['경도']))
fixed_location = list(fixed['location'])

subway_dict = dict(zip(merged_rail['subway'], merged_rail['location']))
subway_location = list(subway_dict.values())
to_subway = haversine_vector(subway_location, fixed_location, comb=True)

close_subway = []
close_distance = []
for i in range(len(to_subway)):
    subway_index = int(np.argmin(to_subway[i]))
    closest_subway = list(subway_dict.keys())[subway_index]
    closest_distance = to_subway[i][subway_index]
    close_subway.append(closest_subway)
    close_distance.append(closest_distance)
    
fixed.loc[:, '지하철역'] = close_subway
fixed.loc[:, '지하철역_거리'] = close_distance
fixed.loc[:, '지하철역_거리'] = fixed['지하철역_거리'].round(4)

In [199]:
rail_list = merged_rail[['역사명', '노선명']]
grouped_rail = rail_list.groupby('역사명')['노선명'].apply(list).reset_index()
grouped_rail.rename(columns={'노선명': '노선명_리스트'}, inplace=True)
grouped_rail

,역사명,노선명_리스트
0,419민주묘지역,[우이신설선]
1,가능역,[1호선]
2,가락시장역,"[3호선, 8호선]"
3,가산디지털단지역,"[7호선, 1호선]"
4,가양역,[9호선]
...,...,...
633,회현역,[4호선]
634,효자역,[의정부선]
635,효창공원앞역,"[6호선, 경의중앙선]"
636,흑석역,[9호선]


In [200]:
fixed['지하철역'] = fixed['지하철역'].apply(lambda x: x.split(" ")[-1])
apartment_rail = pd.merge(fixed, grouped_rail, left_on='지하철역', right_on='역사명', how='inner')

### **3. 주변 의료기관 개수**

병원과의 직선거리 
* 1차 병원 1km 
* 2차 병원 2km 
* 3차 병원 3km

In [201]:
hospital = pd.read_csv("hospital.csv")
hospital = hospital[hospital['location'] != '(0.0, 0.0)']
hospital.loc[:, 'location'] = hospital['location'].apply(ast.literal_eval)

In [202]:
hospital_location_1 = list(hospital[hospital['종별코드명'] == '1차']['location'].unique())
hospital_location_2 = list(hospital[hospital['종별코드명'] == '2차']['location'].unique())
hospital_location_3 = list(hospital[hospital['종별코드명'] == '3차']['location'].unique())

In [203]:
# 1차 병원
distance_to_hospitals = haversine_vector(hospital_location_1, fixed_location, comb=True)

num_close_hospital_1 = []
for distance in distance_to_hospitals:
    number = np.sum(distance <= 1)
    num_close_hospital_1.append(number)
apartment_rail['1차병원'] = num_close_hospital_1

# 2차 병원
distance_to_hospitals = haversine_vector(hospital_location_2, fixed_location, comb=True)

num_close_hospital_2 = []
for distance in distance_to_hospitals:
    number = np.sum(distance <= 2)
    num_close_hospital_2.append(number)

apartment_rail['2차병원'] = num_close_hospital_2

# 3차 병원
distance_to_hospitals = haversine_vector(hospital_location_3, fixed_location, comb=True)

num_close_hospital_3 = []
for distance in distance_to_hospitals:
    number = np.sum(distance <= 3)
    num_close_hospital_3.append(number)
apartment_rail['3차병원'] = num_close_hospital_3

### **4. 주변 공원 개수**

* 전국도시공원정보표준데이터: https://www.data.go.kr/data/15012890/standard.do

In [204]:
park = pd.read_csv('park.csv', encoding='euc-kr')
park_df = park[~park['공원구분'].isin(['묘지공원', '도시농업공원', '가로공원', '공공공지', '기타', '기타공원'])]
park_df.loc[park_df['공원구분'] == '어린인공원', '공원구분'] = '어린이공원'
park_df.loc[park_df['공원구분'] == '도시자연공원구역', '공원구분'] = '도시자연공원'
park_df.loc[park_df['공원구분'] == '도시자연공원구역', '공원구분'] = '도시자연공원'
park_df = park_df[park_df['제공기관명'].str.contains('서울|경기|인천')]
park_df = park_df.dropna(subset=['위도', '경도'])
park_df['location'] = tuple(zip(park_df['위도'], park_df['경도']))
park_df = park_df[['공원명', 'location']]

In [205]:
location = list(apartment_rail['location'])
park_location = list(park_df['location'])

distance_to_park = haversine_vector(park_location, location, comb=True)

num_close_park = []
for distance in distance_to_park:
    number = np.sum(distance <= 1)
    num_close_park.append(number)
    
apartment_rail['공원'] = num_close_park

### **5. 주변 대학 개수**
교육부_전국대학교개황: https://www.data.go.kr/data/15100330/fileData.do

In [206]:
university = pd.read_csv("university.csv")
university_df = university[university['학교구분'].isin(['대학', '전문대학'])]

In [207]:
university['location'] = tuple(zip(university['Latitude'], university['Longitude']))
university_location = list(university['location'])
distance_to_univ = haversine_vector(university_location, location, comb=True)

num_close_univ = []
for distance in distance_to_univ:
    number = np.sum(distance <= 1)
    num_close_univ.append(number)
    
apartment_rail['대학'] = num_close_univ

In [208]:
apartment_rail[apartment_rail['공원'].isna() | apartment_rail['대학'].isna()]

,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,location,지하철역,지하철역_거리,역사명,노선명_리스트,1차병원,2차병원,3차병원,공원,대학


### **6. 광역 / 기초 변수 추가 & 상권 변수 추가** 

In [209]:
apartment_rail.loc[:, '법정동주소'] = apartment_rail['법정동주소'].str.replace(r"\t", "", regex=True)
apartment_rail['광역'] = apartment_rail['법정동주소'].apply(lambda x: x.split(" ")[0])
apartment_rail['광역'].value_counts()

광역
경기도      2044
인천광역시     509
서울특별시     439
서울시       197
인천시       131
안양동         4
광주시         3
Name: count, dtype: int64

In [210]:
apartment_rail.loc[apartment_rail['광역'] == '안양동', '법정동주소'] = '경기도 안양시 만안구 안양동 전파로61번길 20'
apartment_rail.loc[apartment_rail['광역'] == '광주시', '법정동주소'] = '경기도 광주시 탄벌동 532-2번지'

apartment_rail['광역'] = apartment_rail['법정동주소'].apply(lambda x: x.split(" ")[0])
apartment_rail.loc[(apartment_rail['광역'] == '서울시') | (apartment_rail['광역'] == '서울특별시'), '광역'] = '서울'
apartment_rail.loc[(apartment_rail['광역'] == '인천시') | (apartment_rail['광역'] == '인천광역시'), '광역'] = '인천'
apartment_rail.loc[(apartment_rail['광역'] == '경기도'), '광역'] = '경기'

apartment_rail['광역'].value_counts()

광역
경기    2051
인천     640
서울     636
Name: count, dtype: int64

In [211]:
apartment_rail['기초'] = apartment_rail['법정동주소'].apply(lambda x: x.split(" ")[1])
apartment_rail['기초'].unique()

array(['강동구', '강남구', '성남시', '수원시', '광명시', '구로구', '연수구', '동대문구', '서초구',
       '부평구', '안양시', '성북구', '의왕시', '서구', '평택시', '안성시', '용인시', '광주시', '중구',
       '미추홀구', '남동구', '화성시', '이천시', '의정부시', '부천시', '파주시', '구리시', '강북구',
       '계양구', '오산시', '도봉구', '남양주시', '고양시', '양주시', '시흥시', '과천시', '포천시',
       '동작구', '용인처인구', '안산시', '강서구', '은평구', '강화군', '동구', '송파구', '노원구',
       '가평군', '동두천시', '광진구', '중랑구', '종로구', '하남시', '양평군', '김포시', '영등포구',
       '여주시', '관악구', '양천구', '고양덕양구', '연천군', '성동구', '서대문구', '군포시', '성남수정구',
       '용산구', '금천구'], dtype=object)

In [212]:
apartment_rail.loc[apartment_rail['기초'] == '용인처인구', '기초'] = '용인시'
apartment_rail.loc[apartment_rail['기초'] == '성남수정구', '기초'] = '성남시'
apartment_rail.loc[apartment_rail['기초'] == '고양덕양구', '기초'] = '고양시'

In [213]:
apartment_rail['기초'].unique()

array(['강동구', '강남구', '성남시', '수원시', '광명시', '구로구', '연수구', '동대문구', '서초구',
       '부평구', '안양시', '성북구', '의왕시', '서구', '평택시', '안성시', '용인시', '광주시', '중구',
       '미추홀구', '남동구', '화성시', '이천시', '의정부시', '부천시', '파주시', '구리시', '강북구',
       '계양구', '오산시', '도봉구', '남양주시', '고양시', '양주시', '시흥시', '과천시', '포천시',
       '동작구', '안산시', '강서구', '은평구', '강화군', '동구', '송파구', '노원구', '가평군',
       '동두천시', '광진구', '중랑구', '종로구', '하남시', '양평군', '김포시', '영등포구', '여주시',
       '관악구', '양천구', '연천군', '성동구', '서대문구', '군포시', '용산구', '금천구'],
      dtype=object)

#### 상권_변수

In [214]:
area = pd.read_csv("상권_변수.csv")
area.head()

,광역시도,시군구,소매,음식,교육,장례식장,보건의료,유원지오락,총인구수
0,서울,종로구,"9,208","6,184",975,11,198,246,"141,223"
1,서울,중구,"12,053","5,521",587,18,253,204,"120,317"
2,서울,용산구,"9,175","4,830",856,9,137,176,740
3,서울,성동구,"9,516","4,310","1,034",7,171,276,"280,707"
4,서울,광진구,"10,989","4,955","1,016",7,203,556,"337,258"


In [215]:
merged = pd.merge(apartment_rail, area, left_on=['광역', '기초'], right_on=['광역시도', '시군구'], how='inner')

In [216]:
merged = merged[['아파트명', '법정동주소', '위도', '경도', '세대수', '임대세대수', '최고층', '최저층', '최대공급면적',
       '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적', '전용면적', '전용율',
       '방수', '욕실수', '현관구조', '입주예정연도', '공급액(만원)', '대형건설사', '지하철역', '지하철역_거리', 
       '역사명', '노선명_리스트', '1차병원', '2차병원', '3차병원', '공원', '대학', '광역', '기초', 
       '소매', '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수']]

##### 추가적인 전처리

- 용적률과 건폐율이 바뀜
- 세대평균 주차대수가 잘못된 값 수정

In [217]:
merged.loc[merged['건폐율'] > 100, ['용적률','건폐율']] = merged.loc[merged['건폐율'] > 100, ['건폐율', '용적률']].values
merged[merged['아파트명'] == '가평 센트럴파크 더 스카이'][['아파트명', '용적률', '건폐율']] 

,아파트명,용적률,건폐율
3017,가평 센트럴파크 더 스카이,482.0,66.0
3018,가평 센트럴파크 더 스카이,482.0,66.0
3019,가평 센트럴파크 더 스카이,482.0,66.0
3020,가평 센트럴파크 더 스카이,482.0,66.0
3021,가평 센트럴파크 더 스카이,482.0,66.0
3022,가평 센트럴파크 더 스카이,482.0,66.0
3023,가평 센트럴파크 더 스카이,482.0,66.0
3024,가평 센트럴파크 더 스카이,482.0,66.0


In [218]:
merged[merged['세대평균_주차대수']>=3][['아파트명', '세대평균_주차대수']]

,아파트명,세대평균_주차대수
92,고덕 아르테스 미소지움,3.3
93,고덕 아르테스 미소지움,3.3
94,고덕 아르테스 미소지움,3.3
786,안양 광신프로그레스 리버뷰,6.3
787,안양 광신프로그레스 리버뷰,6.3
788,안양 광신프로그레스 리버뷰,6.3
789,안양 광신프로그레스 리버뷰,6.3
790,평촌 트리지아,3.3
791,평촌 트리지아,3.3
792,평촌 트리지아,3.3


In [219]:
merged.loc[merged['아파트명'] == '대곡역 롯데캐슬 엘클라씨', '세대평균_주차대수'] = 1.2
merged.loc[merged['아파트명'] == '고덕 아르테스 미소지움', '세대평균_주차대수'] = 1.2
merged.loc[merged['아파트명'] == '안양 광신프로그레스 리버뷰', '세대평균_주차대수'] = 1.17
merged.loc[merged['아파트명'] == '평촌 트리지아', '세대평균_주차대수'] = 1.26
merged.loc[merged['아파트명'] == '자양 하늘채 베르', '세대평균_주차대수'] = 0.98

### **7. 투기과열지구 및 조정대상지역 & 분양가 상한제 적용지역 여부**

* 2023년 1월 3일 기점 투기과열지구 및 조정대상지역 & 분양가 상한제 적용지역 변화
* 따라서 2023년 1월부터 청약이 진행된 아파트의 경우 1월 3일 변경 후 기준 적용
* 2022년까지 청약이 진행된 아파트의 경우 1월 3일 변경 전 기준 적용

**2023년 1월 3일 변경 후 투기과열지구 및 조정대상지역 & 분양가 상한제 적용지역**
* 서울특별시 송파구 / 강남구 / 서초구 / 용산구로 동일

In [220]:
merged.loc[merged['기초'].isin(['송파구', '강남구', '서초구', '용산구']), '투기과열지구_after'] = 1
merged.loc[~merged['기초'].isin(['송파구', '강남구', '서초구', '용산구']), '투기과열지구_after'] = 0
merged.loc[merged['기초'].isin(['송파구', '강남구', '서초구', '용산구']), '분양가상한제_after'] = 1
merged.loc[~merged['기초'].isin(['송파구', '강남구', '서초구', '용산구']),'분양가상한제_after'] = 0

**2023년 1월 3일 변경 전 투기과열지구 및 조정대상지역**
* 서울특별시 전체 구
* 경기도 과천시 / 광명시 / 하남시 / 성남시(분당/수정구)
* 2023년 이전 분양받은 아파트만 적용

In [221]:
apartment_after_2023 = ['청계SKVIEW', '래미안라그란데', '수유시그니티', '롯데캐슬이스트폴', '강동중앙하이츠시티', '용산호반써밋에이디션', '둔촌현대수린나', '청량리롯데캐슬하이루체', \
                        '서울대벤처타운역푸르지오', 'DMC가재울아이파크', '새절역두산위브트레지움', '엘리프미아역2단지', '엘리프미아역1단지', '휘경자이디센시아', '센트레빌아스테리움시그니처', \
                        '영등포자이디그니티', '포레나인천학익', '인천용현경남아너스빌', '인천연희공원호반써밋파크에디션', '서희스타힐스스타디움센트럴씨티', '인천검단신도시AB19블럭호반써밋', \
                        '미추홀루브루숭의', '칸타빌더스위트', '왕길역금호어울림에듀그린', '검단신도시금강펜테리움3차센트럴파크', '송도역경남아너스빌', '더샵아르테', '진위역서희스타힐스더파크뷰', \
                        '효성해링턴플레이스목감역', '평택고덕국제신도시A-49블록호반써밋3차', '광명센트럴아이파크', '이천중리우미린트리쉐이드', '시흥롯데캐슬시그니처2블록', '시흥롯데캐슬시그니처1블록',\
                        '평택브레인시티대광로제비앙모아엘가', '이안평택안중역', 'e편한세상동탄파크아너스A56블럭','지제역반도체밸리제일풍경채2블록', '오산세교하우스토리더센트럴', '부천역청담더마크', \
                        '청광플러스원', '운정자이시그니처', '인덕원퍼스비엘', '시흥센트럴헤센', '중앙하이츠금광프리미엄아파트', '시화MTV푸르지오디오션', '광명자이더샵포레나', '해링턴플레이스다산파크', \
                        'e편한세상용인역플랫폼시티', '해링턴플레이스진사(2BL)', '해링턴플레이스진사(1BL)', '봉담중흥S-클래스센트럴에듀', '동탄신도시금강펜테리움6차센트럴파크(A59블럭)', '이안시그니처역곡', \
                        '동탄파크릭스A55BL', '파주운정신도시디에트르센트럴', '평택화양서희스타힐스센트럴파크', '고덕자이센트로', 'e편한세상동탄파크아너스A56블럭', '힐스테이트평택화양', \
                        '구리역롯데캐슬시그니처', '수원성중흥S-클래스', '회천2차대광로제비앙센트럴']
merged['아파트명'] = merged['아파트명'].str.replace(' ', '')

In [222]:
apartment_before_2023_set = set(list(merged['아파트명'].unique())) - set(apartment_after_2023)
apartment_before_2023 = list(apartment_before_2023_set)

In [223]:
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['광역'] == '서울'), '투기과열지구_before'] = 1
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['기초'].isin(['과천시', '광명시', '하남시'])), '투기과열지구_before'] = 1
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['기초'] == '성남시') & merged['법정동주소'].str.contains('분당|수정'), '투기과열지구_before'] = 1
merged['투기과열지구_before'] = merged['투기과열지구_before'].fillna(0)

In [224]:
merged['투기과열지구_before'].value_counts()

투기과열지구_before
0.0    2665
1.0     662
Name: count, dtype: int64

**2023년 1월 3일 변경 전 분양가상한제 적용지역**
* 서울특별시: 강남구, 서초구, 송파구, 강동구, 영등포구, 마포구, 성동구, 동작구, 양천구, 용산구, 중구, 광진구, 서대문구  
    * 강서구(5개동)  : 방화동, 공항동, 마곡동, 등촌동, 화곡동   
    * 노원구(4개동)  : 상계동, 중계동, 하계동, 월계동  
    * 동대문구(8개동) : 제기동, 회기동, 이문동, 휘경동, 청량리동, 답십리동, 전농동, 용두동 
    * 성북구(13개동) : 정릉동, 성북동, 동소문동(2/3가), 삼선동(1/2/3가), 보문동(1가), 안암동(3가), 동선동(4가), 돈암동, 길음동, 장위동    
    * 은평구(7개동) : 갈현동, 불광동, 대조동, 역촌동, 신사동, 증산동, 수색동   
* 경기도
    * 과천시(5개동) : 별양동, 부림동, 원문동, 주암동, 중앙동  
    * 하남시(4개동) : 창우동, 신장동, 덕풍동, 풍산동  
    * 광명시(4개동) : 광명동, 소하동, 철산동', 하안동  

In [225]:
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['광역'] == '서울') & (merged['기초'].isin(['강남구', '서초구', '송파구', '강동구', '영등포구', '마포구', '성동구', '동작구',\
                                                              '양천구', '용산구', '중구', '광진구', '서대문구'])), '분양가상한제_before'] = 1
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['기초'] == '강서구') & (merged['법정동주소'].str.contains('방화|공항동|마곡|등촌|화곡')), '분양가상한제_before'] = 1
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['기초'] == '노원구') & (merged['법정동주소'].str.contains('상계|중계|하계|월계')), '분양가상한제_before'] = 1
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['기초'] == '동대문구') & (merged['법정동주소'].str.contains('제기|회귀|이문|휘경|청량리|답십리|전농|용두')), '분양가상한제_before'] = 1
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['기초'] == '성북구') & (merged['법정동주소'].str.contains('정릉|성북|동소문|삼선|보문|안암|동선|돈암|길음|장위')), '분양가상한제_before'] = 1

In [226]:
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['기초'] == '과천시') & (merged['법정동주소'].str.contains('별양|부림|원문|주암|중앙')), '분양가상한제_before'] = 1
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['기초'] == '하남시') & (merged['법정동주소'].str.contains('창우|신장|덕풍|풍산')), '분양가상한제_before'] = 1
merged.loc[(merged['아파트명'].isin(apartment_before_2023)) & (merged['기초'] == '광명시') & (merged['법정동주소'].str.contains('광명|소하|철산|하안')), '분양가상한제_before'] = 1

In [227]:
merged['분양가상한제_before'] = merged['분양가상한제_before'].fillna(0)
merged['분양가상한제_before'].value_counts()

분양가상한제_before
0.0    2987
1.0     340
Name: count, dtype: int64

아래는 확인

In [228]:
# 2023년 이전 투기과열지구였던 과천시/하남시/광명시/성남시(수정/분당) 아파트 중에 2023년 이후 분양 아파트에 속한 아파트가 있는지 확인
apartments_matching_condition = merged[(merged['기초'].isin(['과천시', '광명시', '하남시', '성남시'])) & (merged['투기과열지구_before'] == 1)]['아파트명'].unique()
non_matching_apartments = [apartment for apartment in apartments_matching_condition if apartment in apartment_after_2023]
non_matching_apartments

[]

In [229]:
# 2023년 이전 투기과열지구였던 성남시 아파트 중 수정구 / 분당구가 아닌 아파트가 포함됐는지 주소 확인
merged[(merged['기초'].isin(['성남시'])) & (merged['투기과열지구_before'] == 1)][['아파트명', '법정동주소']].drop_duplicates()

,아파트명,법정동주소
140,산성역자이푸르지오(성남),경기도 성남시 수정구 신흥동 1132
165,위례자이더시티,경기도 성남시 수정구 창곡동 512
173,판교밸리자이1단지,경기도 성남수정구 고등동 582
174,판교밸리자이2단지,경기도 성남수정구 고등동 585
178,판교밸리자이3단지,경기도 성남수정구 고등동 601


In [230]:
# 2023년 이전 분양가상한제 적용을 받은 강서구/노원구/동대문구/성북구/은평구 아파트 중에 2023년 이후 분양 아파트에 속한 아파트가 있는지 확인
apartments_matching_condition = merged[(merged['기초'].isin(['강서구', '노원구', '동대문구', '성북구', '은평구'])) & (merged['분양가상한제_before'] == 1)]['아파트명'].unique()
non_matching_apartments = [apartment for apartment in apartments_matching_condition if apartment in apartment_after_2023]
non_matching_apartments

[]

In [231]:
# 2023년 이전 분양가상한제 적용을 받은 광명시/하남시/과천시 아파트 중에 2023년 이후 분양 아파트에 속한 아파트가 있는지 확인
apartments_matching_condition = merged.loc[(merged['기초'].isin(['광명시', '하남시', '과천시'])) & (merged['분양가상한제_before'] == 1), '법정동주소'].unique()
non_matching_apartments = [apartment for apartment in apartments_matching_condition if apartment in apartment_after_2023]
non_matching_apartments

[]

### **8. 타시군구 통근인구/시군구내 통근인구 변수 추가**

In [232]:
commute = pd.read_csv("commute.csv")

In [233]:
merged = pd.merge(merged, commute, left_on=['광역', '기초'], right_on=['광역', '기초'], how='inner')

### **9. 평당공급액 변수 추가**

In [234]:
merged['평당_공급액'] = (merged['공급액(만원)'] / merged['공급면적']) * 3.3

In [235]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3327 entries, 0 to 3326
Data columns (total 49 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   아파트명           3327 non-null   object 
 1   법정동주소          3327 non-null   object 
 2   위도             3327 non-null   float64
 3   경도             3327 non-null   float64
 4   세대수            3327 non-null   int64  
 5   임대세대수          3093 non-null   float64
 6   최고층            3297 non-null   float64
 7   최저층            3284 non-null   float64
 8   최대공급면적         3327 non-null   float64
 9   최소공급면적         3327 non-null   float64
 10  총아파트동수         3303 non-null   float64
 11  용적률            3327 non-null   float64
 12  건폐율            3327 non-null   float64
 13  세대평균_주차대수      3259 non-null   float64
 14  공급면적           3327 non-null   float64
 15  전용면적           3327 non-null   float64
 16  전용율            3327 non-null   float64
 17  방수             3245 non-null   float64
 18  욕실수     

### **타입 변경**

<object -> int>   
소매 / 음식 / 교육 / 보건의료 / 유원지오락 / 총인구수 / 시군구내_통근통학 / 타시군구_통근통학

In [236]:
columns_to_convert = ['소매', '음식', '교육', '보건의료', '유원지오락', '총인구수', '시군구내_통근통학', '타시군구_통근통학']
merged[columns_to_convert] = merged[columns_to_convert].apply(lambda x: x.str.replace(",", ""))
merged[columns_to_convert] = merged[columns_to_convert].astype(int)

In [237]:
merged.to_csv("apartment_20230830.csv", encoding='UTF-8', index=False)